In [1]:
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd

from scipy import integrate
from scipy.interpolate import interpn,interp1d,RegularGridInterpolator, Akima1DInterpolator
from scipy.optimize import minimize
import scipy.signal as sig
from scipy.optimize import least_squares, minimize

import time
from tqdm import notebook
import importlib

from sklearn.metrics import mean_absolute_error

from numpy import linalg as la

import copy

import sys
from importlib import reload,import_module
import os

#import sagemaker
#import boto3

%matplotlib inline

import copy
import random

#from sources.utilities.model_LTI_v2 import *
#from sources.utilities.utility_functions import *

In [2]:
from treelib import Node, Tree

In [3]:
def discretise_observation(observation):
    '''On discretise les observations entre 0 et 100
    apres les avoir ramenees dans [0, 1]'''
    
    
    position, velocity = observation
    
    position += 1.2
    position /= 1.8
    position = int(100.0 * position)
    
    velocity += -0.07
    velocity /= -0.14
    velocity = int(100.0 * velocity)
    
    return position, velocity

In [4]:
import gym
env = gym.make('MountainCar-v0')
observation = env.reset()

columns = ['Position', 'Velocity', 'Angle','Angular Velocity']

for _ in range(100):
    env.render()
    action = env.action_space.sample()
    #print(action)
    for i in range(10):
        observation, reward, done, info = env.step(action)

        print(discretise_observation(observation))

        if done:
            observation = env.reset()
env.close()

(42, 51)
(42, 52)
(42, 53)
(41, 54)
(41, 55)
(40, 57)
(40, 58)
(39, 59)
(38, 59)
(38, 60)
(37, 60)
(36, 59)
(35, 58)
(35, 57)
(34, 56)
(34, 55)
(33, 54)
(33, 53)
(33, 52)
(33, 51)
(33, 51)
(33, 52)
(32, 52)
(32, 52)
(32, 52)
(32, 53)
(31, 53)
(31, 53)
(31, 53)
(31, 53)
(30, 53)
(30, 52)
(30, 51)
(30, 51)
(30, 50)
(30, 49)
(30, 49)
(30, 48)
(30, 47)
(30, 47)
(31, 46)
(31, 45)
(31, 45)
(32, 44)
(32, 44)
(33, 43)
(33, 43)
(34, 43)
(34, 42)
(35, 42)
(36, 42)
(36, 42)
(37, 42)
(37, 42)
(38, 42)
(39, 42)
(39, 42)
(40, 42)
(40, 42)
(41, 43)
(41, 43)
(42, 43)
(42, 44)
(43, 44)
(43, 45)
(43, 45)
(44, 46)
(44, 47)
(44, 47)
(44, 48)
(44, 49)
(44, 49)
(44, 50)
(44, 51)
(44, 51)
(44, 52)
(44, 53)
(43, 53)
(43, 54)
(43, 54)
(42, 55)
(42, 55)
(41, 56)
(41, 56)
(40, 57)
(40, 57)
(39, 57)
(38, 57)
(38, 57)
(37, 57)
(37, 57)
(36, 56)
(36, 55)
(35, 54)
(35, 53)
(35, 53)
(35, 52)
(34, 51)
(34, 50)
(35, 49)
(35, 48)
(35, 48)
(35, 48)
(35, 48)
(35, 48)
(35, 47)
(35, 47)
(36, 47)
(36, 47)
(36, 47)
(36, 46)
(

(34, 44)
(34, 45)
(35, 45)
(35, 46)
(35, 46)
(35, 47)
(36, 47)
(36, 48)
(36, 48)
(36, 49)
(36, 49)
(36, 49)
(36, 49)
(36, 48)
(36, 48)
(36, 48)
(36, 48)
(37, 48)
(37, 48)
(37, 48)


In [5]:
tree = Tree()
encountered_states = set()

In [6]:
tree.create_node(identifier="root")  # root node

Node(tag=root, identifier=root, data=None)

In [7]:
tree.show()


root



In [8]:
def execute_n_actions(env, nb_exec, action):
    for i in range(nb_exec):
        observation, reward, done, info = env.step(action)

        #print(discretise_observation(observation))

        if done: # or int(reward)==0:
            print(discretise_observation(observation))
            break
    return env, observation, reward, done

In [9]:
tree = Tree()
env = gym.make('MountainCar-v0')

has_reached_goal = False
observation = env.reset()
initial_state = discretise_observation(observation)
initial_env = copy.deepcopy(env.unwrapped)
root = tree.create_node(identifier=discretise_observation(observation), data=(env, 1))
level_nodes = [root]

actions_per_step = 10
goal_reached = None

level = 0

while not has_reached_goal and level_nodes != []:
    print('building childs of level', level)
    next_level_nodes = []
    for node in level_nodes:
        for action in range(3):
            #print('action', action)
            env = copy.deepcopy(node.data[0].unwrapped)
            
            env, observation, reward, done = execute_n_actions(env, actions_per_step, action)
            
            discret_obs = discretise_observation(observation)
            
            if done: # goal reached
                print('Goal reached!')
                has_reached_goal = True
                goal_reached = discret_obs
                
                
            if not tree.contains(discret_obs):
            
                child = tree.create_node(identifier=discret_obs, parent=node, data=(env, action))
                
                if not done:
                    next_level_nodes.append(child)
                
    level_nodes = next_level_nodes
    level += 1


building childs of level 0
building childs of level 1
building childs of level 2
building childs of level 3
building childs of level 4
building childs of level 5
building childs of level 6
building childs of level 7
building childs of level 8
building childs of level 9
(94, 41)
Goal reached!
(95, 39)
Goal reached!
(94, 36)
Goal reached!
(94, 32)
Goal reached!


In [10]:
tree.show()

(41, 50)
├── (37, 59)
│   ├── (27, 62)
│   │   ├── (19, 57)
│   │   │   ├── (17, 49)
│   │   │   │   ├── (21, 41)
│   │   │   │   │   ├── (30, 37)
│   │   │   │   │   │   └── (44, 28)
│   │   │   │   │   │       └── (56, 40)
│   │   │   │   │   ├── (33, 31)
│   │   │   │   │   │   ├── (44, 39)
│   │   │   │   │   │   ├── (47, 33)
│   │   │   │   │   │   │   ├── (52, 52)
│   │   │   │   │   │   │   └── (58, 39)
│   │   │   │   │   │   └── (50, 26)
│   │   │   │   │   │       ├── (59, 48)
│   │   │   │   │   │       ├── (62, 42)
│   │   │   │   │   │       └── (65, 35)
│   │   │   │   │   │           ├── (66, 60)
│   │   │   │   │   │           ├── (69, 52)
│   │   │   │   │   │           └── (72, 45)
│   │   │   │   │   │               ├── (65, 70)
│   │   │   │   │   │               ├── (68, 62)
│   │   │   │   │   │               └── (71, 55)
│   │   │   │   │   └── (35, 24)
│   │   │   │   │       ├── (51, 37)
│   │   │   │   │       │   ├── (52, 57)
│   │   │   │   │       │   │   ├

In [11]:
optimal_path = [path for path in tree.paths_to_leaves() if path[-1] == goal_reached][0]
optimal_path

[(41, 50),
 (42, 46),
 (40, 58),
 (30, 64),
 (20, 60),
 (15, 52),
 (23, 29),
 (45, 18),
 (68, 25),
 (83, 33),
 (94, 32)]

In [12]:
initial_state

(41, 50)

In [13]:
optimal_actions_suit = [tree.get_node(nid).data[1] for nid in optimal_path[1:]]# for _ in range(actions_per_step)]
optimal_actions_suit

[2, 0, 0, 0, 0, 2, 2, 2, 2, 2]

In [14]:
env = copy.deepcopy(initial_env.unwrapped)
env.render()
for action in optimal_actions_suit:

    print(action)
    for i in range(actions_per_step):
        #time.sleep(0.1)
        env.render()
        observation, reward, done, info = env.step(action)
    print(discretise_observation(observation))

        

env.close()

    

2
(42, 46)
0
(40, 58)
0
(30, 64)
0
(20, 60)
0
(15, 52)
2
(23, 29)
2
(45, 18)
2
(68, 25)
2
(83, 33)
2
(96, 32)


In [ ]:
discretise_observation((0.5, 0.0))

In [ ]:
max([a for (a, b) in [l.identifier for l in tree.leaves()]])

In [ ]:
level_nodes

In [ ]:
goal_reached